In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from rtmag.process.paper.utils import get_label_xyz

from rtmag.process.paper.load import MyModel
device = "cpu"
import rtmag.process.paper.metric as m
from rtmag.process.paper.potential_field import get_potential_field
from rtmag.process.paper.opt import relax
from rtmag.process.paper.opth import draw

In [ ]:
isee_reference = "F:/isee_dataset/11158/label"
isee_reference = Path(isee_reference)
isee_reference_files = sorted(isee_reference.glob("*.npz"))


idx = 13
ref_path = isee_reference_files[idx]
print(ref_path.stem)
B, x, y, z = get_label_xyz(ref_path)

In [ ]:
np.save('isee.npy', B)

In [ ]:
draw(B, radius=1)

# Potential

In [ ]:
B = np.load('isee.npy')
B.shape

In [ ]:
bottom = B[:, :, 0, :]
print(bottom.shape)

nx, ny, nz, _ = B.shape
print(nx, ny, nz)

In [ ]:
bp = get_potential_field(bottom[..., 2], nz, l=0.5)

In [ ]:
np.save("pot.npy", bp)

In [ ]:
draw(bp, radius=1)

In [ ]:
res = m.evaluate(bp, B)
print()
print(f"sigmaJ_b: {res['sigmaJ_b']:.4f}")
print(f"div_b   : {res['div_b']:.4f}")
print(f"sigmaJ_B: {res['sigmaJ_B']:.4f}")
print(f"div_B   : {res['div_B']:.4f}")

# Optimization

In [ ]:
B = np.load('isee.npy')
bottom = B[:, :, 0, :]
print(bottom.shape)

In [ ]:
bp = np.load("pot.npy")
bp.shape

In [ ]:
bp[:, :, 0, :] = bottom

In [ ]:
br = relax(bp, 'opt')

In [ ]:
np.save("opt.npy", br)

In [ ]:
draw(br, radius=1)

In [ ]:
res = m.evaluate(br, B)
print()
print(f"sigmaJ_b: {res['sigmaJ_b']:.4f}")
print(f"div_b   : {res['div_b']:.4f}")
print(f"sigmaJ_B: {res['sigmaJ_B']:.4f}")
print(f"div_B   : {res['div_B']:.4f}")

# AI

In [ ]:
meta_path = "D:/models/uno/best_model.pt"

mm = MyModel(meta_path, device=device)

In [ ]:
B = np.load('isee.npy')
B.shape

In [ ]:
b_bottom = B[:, :, 0, :]
b_bottom = b_bottom.transpose((1, 0, 2))[None, None, ...]
print(b_bottom.shape)

In [ ]:
b = mm.get_pred_from_numpy(b_bottom)

In [ ]:
np.save("pino.npy", b)

In [ ]:
draw(b, radius=1)

In [ ]:
res = m.evaluate(b, B)
print()
print(f"sigmaJ_b: {res['sigmaJ_b']:.4f}")
print(f"div_b   : {res['div_b']:.4f}")
print(f"sigmaJ_B: {res['sigmaJ_B']:.4f}")
print(f"div_B   : {res['div_B']:.4f}")

# AI + Optimization

In [ ]:
B = np.load('isee.npy')
b = np.load("pino.npy")

In [ ]:
bottom = B[:, :, 0, :]
print(bottom.shape)

In [ ]:
b[:, :, 0, :] = bottom

In [ ]:
br_pino = relax(b, 'pino_opt')

In [ ]:
np.save("pino_opt.npy", br_pino)

In [ ]:
draw(br_pino, radius=1)

In [ ]:
res = m.evaluate(br_pino, B)
print()
print(f"sigmaJ_b: {res['sigmaJ_b']:.4f}")
print(f"div_b   : {res['div_b']:.4f}")
print(f"sigmaJ_B: {res['sigmaJ_B']:.4f}")
print(f"div_B   : {res['div_B']:.4f}")